In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"  
import pandas as pd
import numpy as np
import wandb                                   
from sklearn.model_selection import GroupShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,precision_recall_fscore_support
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)

In [ ]:
wandb.init(project="LLM-Authorship-Attribution", name="RoBERTa-Master-Run-optimized") 

In [ ]:
MODEL_CHECKPOINT = "roberta-large" 
MAX_LEN = 512
OUTPUT_DIR = "./my_finetuned_models/roberta_author_attribution_all_domain_optimized"
CACHE_DIR = "./hf_cache" 
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(CACHE_DIR, exist_ok=True)

LABEL_MAP = {
    "Llama-3.1": 0,
    "Qwen-2.5": 1,
    "Mistral-v0.3": 2,
    "Granite-3.3": 3,
    "GLM-4": 4

}

num_labels = len(LABEL_MAP)



In [ ]:

print("Load Data")
df_train_full = pd.read_csv("../all_domain_data/train_combined.csv") 
df_val_full   = pd.read_csv("../all_domain_data/val_combined.csv")
df_test_full  = pd.read_csv("../all_domain_data/test_combined.csv")

def fix_columns(df):
    if 'text' in df.columns:
        df.rename(columns={'text': 'source_text'}, inplace=True)

    if 'summary' in df.columns:
        df.rename(columns={'summary': 'text'}, inplace=True)
    
    return df

df_train_full = fix_columns(df_train_full)
df_val_full   = fix_columns(df_val_full)
df_test_full  = fix_columns(df_test_full)

def clean_data(df):
    df = df.dropna(subset=['text']).copy() 
    df['text'] = df['text'].astype(str)

    df = df[df['text'].str.strip() != ""].copy()
    
    return df

print("Cleaning data...")
df_train_full = clean_data(df_train_full)
df_val_full   = clean_data(df_val_full)
df_test_full  = clean_data(df_test_full)

def encode_label(model_name):
    for key, val in LABEL_MAP.items():
        if key.split("-")[0].lower() in str(model_name).lower(): 
            return val
    return -1 

df_train_full['label'] = df_train_full['model'].apply(encode_label)
df_val_full['label']   = df_val_full['model'].apply(encode_label)
df_test_full['label']  = df_test_full['model'].apply(encode_label)

df_train_full = df_train_full[df_train_full['label'] != -1].copy()
df_val_full   = df_val_full[df_val_full['label'] != -1].copy()
df_test_full  = df_test_full[df_test_full['label'] != -1].copy()

In [ ]:
#Tokenization
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_CHECKPOINT, 
    cache_dir=CACHE_DIR 
)
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=MAX_LEN)

hf_train = Dataset.from_pandas(df_train_full)
hf_val   = Dataset.from_pandas(df_val_full)
hf_test  = Dataset.from_pandas(df_test_full)

tokenized_train = hf_train.map(preprocess_function, batched=True)
tokenized_val   = hf_val.map(preprocess_function, batched=True)
tokenized_test  = hf_test.map(preprocess_function, batched=True)

In [ ]:
#Train with wanb
print(f"\n--- 4. Loading Model: {MODEL_CHECKPOINT} ---")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CHECKPOINT, 
    num_labels=num_labels,
    cache_dir=CACHE_DIR 
)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    #  Calculate Accuracy
    acc = accuracy_score(labels, predictions)
    
    # Calculate Precision, Recall, F1 (Macro average)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='macro'
    )
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    report_to="wandb",
    run_name="roberta-run",
    learning_rate=3.85e-5,  
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=64,  
    weight_decay=0.2,            
    num_train_epochs=5,          
    warmup_ratio=0.1,            
    bf16=True,                      
    tf32=True,                      
    dataloader_num_workers=4,       

    
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    logging_dir=f'{OUTPUT_DIR}/logs',
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

print("Starting Training")
trainer.train()

In [ ]:
#evaluation
preds_output = trainer.predict(tokenized_test)
y_pred = np.argmax(preds_output.predictions, axis=1)
y_true = preds_output.label_ids

final_acc = accuracy_score(y_true, y_pred)
print(f"FINAL ACCURACY: {final_acc:.4f}")

# Log final confusion matrix to W&B
wandb.log({"test_accuracy": final_acc})
wandb.sklearn.plot_confusion_matrix(y_true, y_pred, list(LABEL_MAP.keys()))

print("DONE!")

In [ ]:
wandb.finish()